In [12]:
import numpy as np


In [27]:
base_size=16
ratios=[0.5, 1, 2]
base_anchor = np.array([1, 1, base_size, base_size]) - 1   

In [28]:
def _whctrs(anchor):
    """
    Return width, height, x center, and y center for an anchor (window).
    """

    w = anchor[2] - anchor[0] + 1
    h = anchor[3] - anchor[1] + 1
    x_ctr = anchor[0] + 0.5 * (w - 1)
    y_ctr = anchor[1] + 0.5 * (h - 1)
    return w, h, x_ctr, y_ctr

In [29]:
print("width, height, x center, y center:",_whctrs(base_anchor))

width, height, x center, y center: (16, 16, 7.5, 7.5)


In [30]:
w, h, x_ctr, y_ctr =_whctrs(base_anchor)
size = w * h
print("size:",size)
size_ratios = size / ratios
print("size_ratio:",size_ratios)
ws = np.round(np.sqrt(size_ratios))
print("ws:",ws)
hs = np.round(ws * ratios)
print("hs:",hs)

size: 256
size_ratio: [512. 256. 128.]
ws: [23. 16. 11.]
hs: [12. 16. 22.]


In [31]:
ws = ws[:, np.newaxis]
print("ws:",ws)
hs = hs[:, np.newaxis]
print("hs:",hs)
anchors = np.hstack((x_ctr - 0.5 * (ws - 1),
                         y_ctr - 0.5 * (hs - 1),
                         x_ctr + 0.5 * (ws - 1),
                         y_ctr + 0.5 * (hs - 1)))
print("anchors：",anchors)



ws: [[23.]
 [16.]
 [11.]]
hs: [[12.]
 [16.]
 [22.]]
anchors： [[-3.5  2.  18.5 13. ]
 [ 0.   0.  15.  15. ]
 [ 2.5 -3.  12.5 18. ]]


In [52]:
anchors

(3, 4)

In [37]:
def _scale_enum(anchor, scales):
    """
    Enumerate a set of anchors for each scale wrt an anchor.
    """

    w, h, x_ctr, y_ctr = _whctrs(anchor)
    ws = w * scales
    hs = h * scales
    anchors = _mkanchors(ws, hs, x_ctr, y_ctr)
    return anchors

In [50]:
scales=np.array([8,16,32])

In [51]:
_scale_enum(anchors[1,:],scales)

array([[ -56.,  -56.,   71.,   71.],
       [-120., -120.,  135.,  135.],
       [-248., -248.,  263.,  263.]])

In [2]:
import numpy as np
import pdb

# Verify that we compute the same anchors as Shaoqing's matlab implementation:
#
#    >> load output/rpn_cachedir/faster_rcnn_VOC2007_ZF_stage1_rpn/anchors.mat
#    >> anchors
#
#    anchors =
# w:h=2:1
#       -83   -39   100    56
#      -175   -87   192   104
#      -359  -183   376   200
# w:h=1:1
#       -55   -55    72    72
#      -119  -119   136   136
#      -247  -247   264   264
# w:h=1:2
#       -35   -79    52    96
#       -79  -167    96   184
#      -167  -343   184   360

#array([[ -83.,  -39.,  100.,   56.],
#       [-175.,  -87.,  192.,  104.],
#       [-359., -183.,  376.,  200.],
#       [ -55.,  -55.,   72.,   72.],
#       [-119., -119.,  136.,  136.],
#       [-247., -247.,  264.,  264.],
#       [ -35.,  -79.,   52.,   96.],
#       [ -79., -167.,   96.,  184.],
#       [-167., -343.,  184.,  360.]])

try:
    xrange          # Python 2
except NameError:
    xrange = range  # Python 3

# anchor=[x1,y1,x2,y2]
def generate_anchors(base_size=16, ratios=[0.5, 1, 2],
                     scales=2**np.arange(3, 6)): #[8,16,32]
    """
    Generate anchor (reference) windows by enumerating aspect ratios（长宽比） X 
    scales wrt a reference (0, 0, 15, 15) window.
    """

    base_anchor = np.array([1, 1, base_size, base_size]) - 1   # [0,0,15,15]
    # return同一中心点的3种不同比例的anchor [x1,y1,x2,y2] * 3
    ratio_anchors = _ratio_enum(base_anchor, ratios)
    # ratio_anchors.shape =[3,4]
    anchors = np.vstack([_scale_enum(ratio_anchors[i, :], scales)
                         for i in xrange(ratio_anchors.shape[0])])
    return anchors

def _whctrs(anchor):
    """
    中心点
    Return width, height, x center, and y center for an anchor (window).
    """

    w = anchor[2] - anchor[0] + 1
    h = anchor[3] - anchor[1] + 1
    x_ctr = anchor[0] + 0.5 * (w - 1)
    y_ctr = anchor[1] + 0.5 * (h - 1)
    return w, h, x_ctr, y_ctr

def _mkanchors(ws, hs, x_ctr, y_ctr):
    """
    得到anchor
    Given a vector of widths (ws) and heights (hs) around a center
    (x_ctr, y_ctr), output a set of anchors (windows).
    """

    ws = ws[:, np.newaxis]  #shape(3,1)
    hs = hs[:, np.newaxis]
    anchors = np.hstack((x_ctr - 0.5 * (ws - 1),
                         y_ctr - 0.5 * (hs - 1),
                         x_ctr + 0.5 * (ws - 1),
                         y_ctr + 0.5 * (hs - 1)))
    return anchors

def _ratio_enum(anchor, ratios):
    """
    重组不同比例
    Enumerate a set of anchors for each aspect ratio wrt an anchor.
    """
    #保证不同比例的anchors面积相同，小数点误差
    w, h, x_ctr, y_ctr = _whctrs(anchor)
    size = w * h     #256 =16*16
    size_ratios = size / ratios   # [512,256,128]
    ws = np.round(np.sqrt(size_ratios)) #[23. 16. 11.]
    hs = np.round(ws * ratios) #[12. 16. 22.]
    anchors = _mkanchors(ws, hs, x_ctr, y_ctr)
    return anchors

def _scale_enum(anchor, scales):
    """
    不同大小
    Enumerate a set of anchors for each scale wrt an anchor.
    """

    w, h, x_ctr, y_ctr = _whctrs(anchor)
    ws = w * scales
    hs = h * scales
    anchors = _mkanchors(ws, hs, x_ctr, y_ctr)
    return anchors

a = generate_anchors()


In [40]:
a

array([[ -84.,  -40.,   99.,   55.],
       [-176.,  -88.,  191.,  103.],
       [-360., -184.,  375.,  199.],
       [ -56.,  -56.,   71.,   71.],
       [-120., -120.,  135.,  135.],
       [-248., -248.,  263.,  263.],
       [ -36.,  -80.,   51.,   95.],
       [ -80., -168.,   95.,  183.],
       [-168., -344.,  183.,  359.]])

In [9]:
import torch
anchors = torch.from_numpy(a)

In [13]:
_num_anchors=anchors.size(0)

In [18]:
feat_height=10
feat_width=10
feat_stride=16

shift_x = np.arange(0, feat_width) * feat_stride    #shape: [width,] 特征图相对于原图的偏移
shift_y = np.arange(0, feat_height) * feat_stride 
print("shift_x:",shift_x)
print("shift_y:",shift_y)

shift_x: [  0  16  32  48  64  80  96 112 128 144]
shift_y: [  0  16  32  48  64  80  96 112 128 144]


In [29]:
shift_x.ravel()

array([  0,  16,  32,  48,  64,  80,  96, 112, 128, 144,   0,  16,  32,
        48,  64,  80,  96, 112, 128, 144,   0,  16,  32,  48,  64,  80,
        96, 112, 128, 144,   0,  16,  32,  48,  64,  80,  96, 112, 128,
       144,   0,  16,  32,  48,  64,  80,  96, 112, 128, 144,   0,  16,
        32,  48,  64,  80,  96, 112, 128, 144,   0,  16,  32,  48,  64,
        80,  96, 112, 128, 144,   0,  16,  32,  48,  64,  80,  96, 112,
       128, 144,   0,  16,  32,  48,  64,  80,  96, 112, 128, 144,   0,
        16,  32,  48,  64,  80,  96, 112, 128, 144])

In [19]:
shift_x, shift_y = np.meshgrid(shift_x, shift_y)
print("shift_x:",shift_x)
print("shift_y:",shift_y)

shift_x: [[  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]
 [  0  16  32  48  64  80  96 112 128 144]]
shift_y: [[  0   0   0   0   0   0   0   0   0   0]
 [ 16  16  16  16  16  16  16  16  16  16]
 [ 32  32  32  32  32  32  32  32  32  32]
 [ 48  48  48  48  48  48  48  48  48  48]
 [ 64  64  64  64  64  64  64  64  64  64]
 [ 80  80  80  80  80  80  80  80  80  80]
 [ 96  96  96  96  96  96  96  96  96  96]
 [112 112 112 112 112 112 112 112 112 112]
 [128 128 128 128 128 128 128 128 128 128]
 [144 144 144 144 144 144 144 144 144 144]]


In [39]:
shifts = np.vstack((shift_x.ravel(), shift_y.ravel(),shift_x.ravel(),shift_y.ravel()))
shifts_transpose = shifts.transpose()
# print("shifts:",shifts)
shifts_transpose=torch.from_numpy(shifts_transpose)
print("shifts_transpos:",shifts_transpose.size(0))


shifts_transpos: 100


In [64]:
anchors = a.reshape(1, 9, 4) + shifts.reshape(100, 1, 4)

anchors = anchors.reshape(1,9*100,4)
print(anchors)

[[[ -84.  -24.  131.  103.]
  [-176.  -72.  223.  151.]
  [-360. -168.  407.  247.]
  ...
  [ 108.   64.  195.  239.]
  [  64.  -24.  239.  327.]
  [ -24. -200.  327.  503.]]]


In [65]:
anchors = anchors.reshape(1,900,4)

In [67]:
anchors.shape

(1, 900, 4)

In [79]:
a = np.random.rand(2,3,4)

In [80]:
a

array([[[0.35483246, 0.50374436, 0.48460624, 0.31009704],
        [0.33868736, 0.49817436, 0.22757828, 0.27140811],
        [0.12068433, 0.30963172, 0.21534427, 0.9504759 ]],

       [[0.41138889, 0.61004444, 0.56730023, 0.73510276],
        [0.98815535, 0.00859608, 0.5406151 , 0.27182155],
        [0.19928739, 0.08759861, 0.43019506, 0.47275651]]])

In [81]:
a[0]

array([[0.35483246, 0.50374436, 0.48460624, 0.31009704],
       [0.33868736, 0.49817436, 0.22757828, 0.27140811],
       [0.12068433, 0.30963172, 0.21534427, 0.9504759 ]])

In [ ]:
import torch
scores = 